In [ ]:
!python --version

In [ ]:
import os, re
from enum import Enum
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/My Drive/Drones/"
dataframes_folder = path + "dataframes/"

In [ ]:
dataframes = []
for sheet in os.listdir(dataframes_folder):
  dataframes.append(pd.read_csv(dataframes_folder+ sheet))
df = pd.concat(dataframes, ignore_index=True)
df.to_csv(path+"join_drones.csv")
df.shape, df.columns

NameError: name 'os' is not defined

In [ ]:
df.rename(columns = {'Price':'price', 'Sales':'sales', 'Stars':'stars', 'Modelo':'model', 'Marca':'brand', 'Unidades por pack':'units', 'Cantidad de baterías incluidas':
                     'batteries_included', 'Es mini':'mini', 'Tiempo máximo de vuelo':'flight_time', 'Es FPV':'is_fpv', 'Resolución máxima de la cámara': 'resolution',
                     'Con cámara': 'camara', 'Frecuencia de funcionamiento del Wi-Fi':'wifi_freq', 'Con Wi-Fi':'wifi', 'Con GPS':'gps',
                    'Con control remoto':'remote_control', 'Page':'link', 'Línea':'line', 'Versión':'version', 'Color':'color',
                    'Tipos de drone':'type', 'Es plegable':'portable', 'Fabricante':'manufacter', 'Modelo alfanumérico':'alphanumeric',
                    'Ambientes de uso':'territories', 'Con protector de hélices':'helix_protector',
                    'Con función de auto retorno':'autoreturn', 'Con giro 360':'full_turn',
                    'Con retención de altitud':'altitude_retention', 'Temperatura mínima de funcionamiento - Temperatura máxima de funcionamiento': 'temp_range',
                    'Accesorios incluidos':'accesories', 'Con cámara dual':'dual_camara', 'Zoom de la cámara':'zoom',
                    'Velocidad máxima':'speed', 'Resistencia al viento':'wind_resistance',
                    'Máxima distancia de vuelo':'distance', 'Máxima altura de vuelo':'altitude',
                    'Cantidad de motores':'motors', 'Duración de la batería':'battery_life',
                    'Con función de seguimiento':'follow_func', 'Es controlado por smartphone':'phone_driven',
                    'Con modo sin cabeza':'without_head_mode', 'Edad mínima recomendada':'min_age',
                    'Duración del producto':'lifetime', 'Plazo de repuestos o servicio técnico':'warranty',
                    'Tamaño de la pantalla del control remoto':'screen_size', 'Con control remoto táctil':'touch_driven',
                    'Es kit':'is_kit', 'Usos recomendados':'recommended_uses', 'Homologación Anatel Nº':'homologation'}, inplace = True)

data = df

In [ ]:
# Try to filter mystery boxes
filter = "caja|box|mystery|misteriosa|pájaro|liquidacion|paquetes|bird|piezas|toy|juguete|engranes|engranajes"
links = df["link"].tolist()
for index, element in enumerate(links):
  element = str(element)
  if re.search(filter, element, flags= re.IGNORECASE) != None:
    df.drop(index=index, inplace=True)

models = df["model"].tolist()
for index, element in enumerate(models):
  element = str(element)
  if re.search(filter, element, flags= re.IGNORECASE) != None and index in df.index:
      df.drop(index=index, inplace=True)

df.shape

In [ ]:
# Drop empty columns
for column in df.columns:
  if df[column].isna().sum() > 600:
    df.pop(column)
df.columns

NameError: name 'df' is not defined

In [ ]:
#Clean sales:
sales = df["sales"].tolist()
new_sales = []
for element in sales:
  if type(element) == str:
    element  = int(element.replace("mil", "000"))
  new_sales.append(element)
df["sales"] = new_sales

#Clean price
prices = df["price"].tolist()
new_prices = []
for element in prices:
  element = int(element.replace(",", ""))
  new_prices.append(element)
df["price"] = new_prices

not_drones = df.loc[df["price"] < 100]
df.drop(index= not_drones.index, inplace=True)

#Clean model:
df["model"].fillna("Generic", inplace=True)
df["model"].replace("Others", "Generic", inplace=True)

#Clean brand
brand = df["brand"].tolist()
new_brand = []
for element in brand:
  element = element.lower()
  element = re.sub("\W\S", "_", element)
  element = re.sub("[Gg]en[ée]rica", "generic", element)
  element = re.sub("other", "generic", element)
  element = re.sub("[\s\S]*generic[\s\S]*", "generic", element)
  if len(element) > 16:
    element = "generic"
  new_brand.append(element)
df["brand"] = new_brand
df["brand"].fillna("generic", inplace=True)
df.loc[118, "brand"] = "amor_mio"
df.loc[217, "brand"] = "circulo"
df.loc[254, "brand"] = "fotorama"
df.loc[319, "brand"] = "light_moo"
df.loc[405, "brand"] = "generic"
df.loc[502, "brand"] = "light_moo"
df.loc[511, "brand"] = "K&F"
df.loc[541, "brand"] = "generic"
df.loc[555, "brand"] = "guangdong"
df.loc[572, "brand"] = "circulo"
df.loc[603, "brand"] = "generic"

#Clean flight time:
times = df["flight_time"].tolist()
new_times = []
for element in times:
  if type(element) == str:
    element  = element.replace(" ", "")
    element  = element.replace("m", "")
    try:
      float(element)
    except:
      element  = element.replace("h", "")
      element = float(element)*60
  new_times.append(element)
df["flight_time"] = new_times

df["flight_time"] = df["flight_time"].astype("float")
df["flight_time"].fillna(10, inplace=True)
df.loc[df["flight_time"] < 5, "flight_time"] = 5

# Clean distance:
distances = df["distance"].tolist()
new_distances = []
for element in distances:
  if type(element) == str:
    element = element.replace(" ", "")
    element = element.replace("m", "")
    if re.search("k", element) != None:
      element = element.replace("k", "")
      element = float(element) * 1000
    elif re.search("ft", element) != None:
      element = element.replace("ft", "")
      element = float(element) * 0.3048
    elif re.search('\"', element) != None:
      element = element.replace('"', '')
      element = float(element) * 0.0254
    else:
      element = float(element)
    element = round(element)
  new_distances.append(element)
df["distance"] = pd.Series(new_distances)
df["distance"].fillna(200, inplace=True)

#Clean wifi_freq
wifi_freq = df["wifi_freq"].tolist()
new_wifi_freq = []
for element in wifi_freq:
  if type(element) == str:
    element = element.replace(" ", "")
    element = element.replace("GHz", "")
  new_wifi_freq.append(element)
df["wifi_freq"] = new_wifi_freq
df.loc[(df["wifi_freq"].isnull()) & (df["wifi"]==True), "wifi_freq"] = 2.4

In [ ]:
#Clean resolution
NOT, SD, HD, FULL_HD, FOUR_K, FIVE_K, SIX_K, SEVEN_K, EIGHT_K = range(9)

# NOT camara
df["camara"].replace("No", False, inplace = True)
df["camara"].fillna(False, inplace = True)
df.loc[df["camara"]==False, "resolution"] = NOT

#SD camara
df["resolution"].fillna(SD, inplace=True)
df["resolution"].replace("[\s\S]*[Ss][Dd][\s\S]*", SD, regex=True, inplace=True)

#FULL HD camara
df["resolution"].replace("[\s\S]*1080[\s\S]*", FULL_HD, regex=True, inplace=True)
df["resolution"].replace("[\s\S]*[Ff]ull[\s\S]*", FULL_HD, regex=True, inplace=True)
df["resolution"].replace("[\s\S]*2K[\s\S]*", FULL_HD, regex=True, inplace=True)
  #IMPORTANT: Because of the regex pattern 1080px search should be made before 720px search

#HD camara
df["resolution"].replace("[\s\S]*720[\s\S]*", HD, regex=True, inplace=True)
df["resolution"].replace("[\s\S]*[Hh][Dd][\s\S]*", HD, regex=True, inplace=True)

#4K - 8K camara
df["resolution"].replace("[\s\S]*4[\s\S]*", FOUR_K, regex=True, inplace=True)
df["resolution"].replace("[\s\S]*5[\s\S]*", FIVE_K, regex=True, inplace=True)
df["resolution"].replace("[\s\S]*6[\s\S]*", SIX_K, regex=True, inplace=True)
df["resolution"].replace("[\s\S]*7[\s\S]*", SEVEN_K, regex=True, inplace=True)
df["resolution"].replace("[\s\S]*8[\s\S]*", EIGHT_K, regex=True, inplace=True)

In [ ]:
# Change booleans and fill null values.
df.loc[df["batteries_included"].notnull(), "batteries_included"] = True
df["batteries_included"].fillna(False, inplace=True)

df.loc[df["mini"]=="Sí", "mini"] = True
df.loc[df["mini"]=="No", "mini"] = False
df["mini"].fillna(True, inplace=True)

df.loc[df["is_fpv"]=="Sí", "is_fpv"] = True
df.loc[df["is_fpv"]=="No", "is_fpv"] = False
df["is_fpv"].fillna(False, inplace=True)

df["wifi"].replace("Sí", True, inplace=True)
df["wifi"].replace("No", False, inplace= True)
df["wifi"].fillna(False, inplace=True)

df["gps"].replace("Sí", True, inplace=True)
df["gps"].replace("No", False, inplace=True)
df["gps"].fillna(False, inplace=True)

df["remote_control"].replace("Sí", True, inplace=True)
df["remote_control"].replace("No", False, inplace=True)
df["remote_control"].fillna(False, inplace=True)

df["helix_protector"].replace("Sí", True, inplace=True)
df["helix_protector"].replace("No", False, inplace=True)
df["helix_protector"].fillna(False, inplace=True)

df["portable"].replace("Sí", True, inplace=True)
df["portable"].replace("No", False, inplace=True)
df["portable"].fillna(False, inplace=True)

df["autoreturn"].replace("Sí", True, inplace=True)
df["autoreturn"].replace("No", False, inplace=True)
df["autoreturn"].fillna(False, inplace=True)

df["full_turn"].replace("Sí", True, inplace=True)
df["full_turn"].replace("No", False, inplace=True)
df["full_turn"].fillna(False, inplace=True)

df["altitude_retention"].replace("Sí", True, inplace=True)
df["altitude_retention"].replace("No", False, inplace=True)
df["altitude_retention"].fillna(False, inplace=True)

df["dual_camara"].replace("Sí", True, inplace = True)
df["dual_camara"].replace("No", False, inplace = True)
df["dual_camara"].fillna(False, inplace=True)

df["follow_func"].replace("Sí", True, inplace=True)
df["follow_func"].replace("No", False, inplace=True)
df["follow_func"].fillna(False, inplace=True)

df["wifi_freq"].fillna(2.4, inplace=True)
df["stars"].replace(0, 4, inplace = True)


In [ ]:
# Drop useless columns
df.drop("accesories", axis=1, inplace=True)
df.drop("line", axis=1, inplace=True)
df.drop("version", axis=1, inplace=True)
df.drop("type", axis=1, inplace=True)
df.drop("manufacter", axis=1, inplace=True)
df.drop("alphanumeric", axis=1, inplace=True)
df.drop("motors", axis = 1, inplace=True)
df.drop("camara", axis=1, inplace=True)
df.drop("color", axis=1, inplace=True)
df.drop("units", axis=1, inplace=True)
df.drop("model", axis=1, inplace=True)

In [2]:
df["link"] = df["link"].astype("string")
df["brand"] = df["brand"].astype("string")

df["price"]= df["price"].astype("int64")
df["sales"] = df["sales"].astype("int64")
df["stars"] = df["stars"].astype("float")
df["resolution"] = df["resolution"].astype("int32")
df["wifi_freq"] = df["wifi_freq"].astype("float")
df["distance"] = df["distance"].astype("int64")

df["batteries_included"] = df["batteries_included"].astype("bool")
df["mini"] = df["mini"].astype("bool")
df['flight_time'] = df['flight_time'].astype("bool")
df["is_fpv"] = df["is_fpv"].astype("bool")
df["wifi"] = df["wifi"].astype("bool")
df["gps"] = df["gps"].astype("bool")
df["remote_control"] = df["remote_control"].astype("bool")
df["portable"] = df["portable"].astype("bool")
df["helix_protector"] = df['helix_protector'].astype("bool")
df["autoreturn"] = df['autoreturn'].astype("bool")
df["full_turn"] = df["full_turn"].astype("bool")
df["altitude_retention"] = df["altitude_retention"].astype("bool")
df["dual_camara"] = df['dual_camara'].astype("bool")
df["follow_func"] = df['follow_func'].astype("bool")

NameError: name 'df' is not defined

In [ ]:
link = df.pop("link")
df.head(50)

In [ ]:
df.insert(0, "link", link)
brand = df.pop("brand")
df.insert(1, "brand", brand)
df.to_csv(path+"cleaned.csv")